# GabeDA Features (Customer-Daily Activity)

This notebook creates customer activity metrics at the daily granularity.  
It aggregates transactions per customer per day to analyze daily purchasing patterns.

**Input:** Preprocessed transactions from 01_transactions notebook  
**Output:** Customer-daily metrics (1 row per customer per day)  
**Group By:** `customer_id`, `dt_date`

## 1. Setup: Imports, Context Loading, Logging

## 0. Project Root Setup (Auto-generated)

In [1]:
# Auto-detect project root and add to Python path
import os
import sys
from pathlib import Path

# Get the project root (2 levels up from notebooks/development or notebooks/from_store)
notebook_dir = Path.cwd() if '__file__' not in globals() else Path(__file__).parent
project_root = notebook_dir.parent.parent

# Change to project root
os.chdir(project_root)

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Working directory: {os.getcwd()}")
print(f"Project root: {project_root}")

Working directory: c:\Projects\play\khujta_ai_business
Project root: c:\Projects\play\khujta_ai_business


In [2]:
import pandas as pd
import numpy as np

# v2.0 Refactored imports
from src.utils.logger import setup_logging, get_logger
from src.core.context import GabedaContext
from src.core.persistence import load_context_state, get_latest_state, save_context_state
from src.core.constants import *
from src.features.store import FeatureStore
from src.features.resolver import DependencyResolver
from src.features.detector import FeatureTypeDetector
from src.features.analyzer import FeatureAnalyzer
from src.execution.calculator import FeatureCalculator
from src.execution.groupby import GroupByProcessor
from src.execution.executor import ModelExecutor
from src.export.excel import ExcelExporter

# Load latest context state
client_name = 'test_client'
latest_state = get_latest_state(client_name, base_dir='data/context_states')

if latest_state:
    ctx, base_cfg = load_context_state(latest_state)
    print(f"✓ Loaded latest state: {latest_state}")
else:
    raise FileNotFoundError(f"No context state found for client '{client_name}'")

# Setup logging
setup_logging(log_level=base_cfg.get('log_level', 'INFO'), 
              config={'client': base_cfg.get('client', 'unknown_client')})
logger = get_logger(__name__)

print(f"\n✓ Context loaded successfully!")
print(f"  - Original run_id: {ctx.original_run_id}")
print(f"  - New run_id: {ctx.run_id}")
print(f"  - Available datasets: {len(ctx.list_datasets())} datasets")

✓ Loaded latest state: data\context_states\test_client_20251022_150907
📝 Run instance ID: test_client_20251022_151013 - Logging [INFO] to: logs\test_client_20251022_151013.log

✓ Context loaded successfully!
  - Original run_id: test_client_20251022_151001
  - New run_id: test_client_20251022_151013
  - Available datasets: 9 datasets


## 2. Load Input Data

In [3]:
# Get input dataset
input_df = ctx.get_dataset('transactions_filters')

print(f"✓ Input dataset loaded")
print(f"  - Shape: {input_df.shape}")
print(f"  - Date range: {input_df['dt_date'].min()} to {input_df['dt_date'].max()}")
print(f"  - Unique customers: {input_df['in_customer_id'].nunique()}")
print(f"  - Unique days: {input_df['dt_date'].nunique()}")
print(f"\nFirst few rows:")
input_df.head()

✓ Input dataset loaded
  - Shape: (609, 59)
  - Date range: 20251001 to 20251030
  - Unique customers: 15
  - Unique days: 30

First few rows:


,in_dt,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,...,cost_unit,cost_total,price_unit,price_total,margin_unit,margin_unit_pct,margin_unit_valid,margin_total,margin_total_pct,margin_total_valid
0,2025-10-01 01:02:00,prod8,2.0,52964.0,return,client13,product 8,category B,pack,61.0,...,18792.0,37585.0,26482.0,52964.0,7690.0,29.04,True,15379.0,29.04,True
1,2025-10-01 06:24:00,prod4,6.0,177195.0,sale,client6,product 4,category B,unit,30.0,...,21526.0,129155.0,29533.0,177195.0,8007.0,27.11,True,48040.0,27.11,True
2,2025-10-01 08:38:00,prod7,2.0,70492.0,return,client12,product 7,category A,unit,78.0,...,25754.0,51509.0,35246.0,70492.0,9492.0,26.93,True,18983.0,26.93,True
3,2025-10-01 09:59:00,prod2,4.0,86751.0,sale,client3,product 2,category A,unit,80.0,...,12947.0,51786.0,21688.0,86751.0,8741.0,40.30,True,34965.0,40.31,True
4,2025-10-01 10:07:00,prod3,3.0,76465.0,sale,client12,product 3,category B,unit,47.0,...,16943.0,5083.0,25488.0,76465.0,8545.0,33.53,True,71382.0,93.35,True


## 3. Define Features

Customer daily activity metrics:  
- Product diversity (unique products purchased)
- Transaction frequency (number of transactions)
- Quantity metrics (items purchased)
- Spending metrics (total, average, median spend)
- Transaction value (average spend per visit)

In [4]:
# ===== Customer-Daily Activity Metrics =====
# Based on specs: Daily customer purchase patterns

# Identifiers
def product_id_count(product_id):
    """
    Count of unique products purchased by customer on this day.
    Formula: COUNT(DISTINCT product_id)
    """
    return len(np.unique(product_id))

def trans_id_count(trans_id):
    """
    Number of transactions by customer on this day.
    Formula: COUNT(DISTINCT trans_id)
    """
    return len(np.unique(trans_id))

# Quantity
def quantity_sum(quantity):
    """
    Total items purchased by customer on this day.
    Formula: SUM(quantity)
    """
    return np.sum(quantity)

def quantity_mean(quantity):
    """
    Average items per transaction on this day.
    Formula: AVG(quantity)
    """
    return round(np.mean(quantity), 2)

def quantity_median(quantity):
    """
    Median items per transaction on this day.
    Formula: MEDIAN(quantity)
    """
    return round(np.median(quantity), 2)

# Price
def price_total_sum(price_total):
    """
    Total amount spent by customer on this day.
    Formula: SUM(price_total)
    """
    return np.sum(price_total)

def price_total_mean(price_total):
    """
    Average transaction amount on this day.
    Formula: AVG(price_total)
    """
    return round(np.mean(price_total), 2)

def price_total_median(price_total):
    """
    Median transaction amount on this day.
    Formula: MEDIAN(price_total)
    """
    return round(np.median(price_total), 2)

def transaction_mean(price_total_sum, trans_id_count):
    """
    Average spend per visit on this day.
    Formula: price_total_sum / trans_id_count
    """
    if trans_id_count == 0:
        return DEFAULT_FLOAT
    return round(price_total_sum / trans_id_count, 2)

print("✓ Feature functions defined: 9 attributes")

✓ Feature functions defined: 9 attributes


## 4. Configure Model

In [5]:
# Collect features into dictionary
features = {
    'product_id_count': product_id_count,
    'trans_id_count': trans_id_count,
    'quantity_sum': quantity_sum,
    'quantity_mean': quantity_mean,
    'quantity_median': quantity_median,
    'price_total_sum': price_total_sum,
    'price_total_mean': price_total_mean,
    'price_total_median': price_total_median,
    'transaction_mean': transaction_mean,
}

# Model configuration
cfg_model = {
    'model_name': 'customer_daily',
    'input_dataset_name': 'transactions_filters',
    'group_by': ['in_customer_id', 'dt_date'],  # Customer-day granularity
    'row_id': 'in_trans_id',
    'output_cols': list(features.keys()),
    'features': features,
}

print(f"✓ Model configured: '{cfg_model['model_name']}'")
print(f"  - Group by: {cfg_model['group_by']}")
print(f"  - Output features: {len(cfg_model['output_cols'])}")

✓ Model configured: 'customer_daily'
  - Group by: ['in_customer_id', 'dt_date']
  - Output features: 9


## 5. Prepare Features (Store, Resolve Dependencies, Save Config)

In [6]:
# Initialize feature store and store features
feature_store = FeatureStore()
feature_store.store_features(features, model_name=cfg_model['model_name'], auto_save=True)

# Resolve dependencies
resolver = DependencyResolver(feature_store)
in_cols, exec_seq, ext_cols = resolver.resolve_dependencies(
    output_cols=cfg_model['output_cols'],
    available_cols=input_df.columns.tolist(),
    group_by=cfg_model.get('group_by'),
    model=cfg_model['model_name']
)

# Update model config with resolved dependencies
cfg_model['in_cols'] = in_cols
cfg_model['exec_seq'] = exec_seq
cfg_model['ext_cols'] = ext_cols

# Save master configuration
feature_store.save_master_config(
    model_name=cfg_model['model_name'],
    model_config=cfg_model
)

print("✓ Features prepared and dependencies resolved")
print(f"  - Input columns needed: {len(in_cols)}")
print(f"  - Execution sequence: {exec_seq}")
print(f"  - Master config saved: feature_store/{cfg_model['model_name']}/master_cfg.json")

✓ Features prepared and dependencies resolved
  - Input columns needed: 4
  - Execution sequence: ['product_id_count', 'trans_id_count', 'quantity_sum', 'quantity_mean', 'quantity_median', 'price_total_sum', 'price_total_mean', 'price_total_median', 'transaction_mean']
  - Master config saved: feature_store/customer_daily/master_cfg.json


## 6. Execute Model (Initialize Components + Execute + Store Results)

In [7]:
# Initialize execution components
detector = FeatureTypeDetector()
analyzer = FeatureAnalyzer(feature_store, detector)
calculator = FeatureCalculator()
groupby_processor = GroupByProcessor(calculator, detector)
executor = ModelExecutor(analyzer, groupby_processor, context=ctx)

# Execute model
output = executor.execute_model(
    cfg_model=cfg_model,
    input_dataset_name=cfg_model['input_dataset_name']
)

# Store results in context
ctx.set_model_output(cfg_model['model_name'], output, cfg_model)

print("✓ Model executed successfully!")
print(f"  - Filters: {output['filters'].shape if output['filters'] is not None else 'None'}")
print(f"  - Attributes: {output['attrs'].shape if output['attrs'] is not None else 'None'}")
print(f"  - Customer-days profiled: {output['attrs'].shape[0] if output['attrs'] is not None else 0}")

✓ Model executed successfully!
  - Filters: (609, 59)
  - Attributes: (327, 11)
  - Customer-days profiled: 327


## 7. View Results

In [8]:
# View customer-daily activity (aggregated attributes)
attrs = ctx.get_model_attrs(cfg_model['model_name'])
print(f"Customer-Daily Activity Metrics (n={len(attrs)})")
print(f"Date range: {attrs['dt_date'].min()} to {attrs['dt_date'].max()}")
print(f"Unique customers: {attrs['in_customer_id'].nunique()}")
attrs.head(10)

Customer-Daily Activity Metrics (n=327)
Date range: 20251001 to 20251030
Unique customers: 15


,in_customer_id,dt_date,product_id_count,trans_id_count,quantity_sum,quantity_mean,quantity_median,price_total_sum,price_total_mean,price_total_median,transaction_mean
0,client1,20251002,1,1,1,1.00,1.0,27561.0,27561.00,27561.0,27561.00
1,client1,20251003,2,2,4,2.00,2.0,73943.0,36971.50,36971.5,36971.50
2,client1,20251004,3,3,8,2.67,2.0,245080.0,81693.33,69696.0,81693.33
3,client1,20251005,2,2,17,8.50,8.5,377665.0,188832.50,188832.5,188832.50
4,client1,20251008,1,1,1,1.00,1.0,16346.0,16346.00,16346.0,16346.00
5,client1,20251009,1,1,4,4.00,4.0,97775.0,97775.00,97775.0,97775.00
6,client1,20251010,2,3,9,3.00,3.0,229071.0,76357.00,79776.0,76357.00
7,client1,20251011,2,2,8,4.00,4.0,282823.0,141411.50,141411.5,141411.50
8,client1,20251012,1,1,7,7.00,7.0,14937.0,14937.00,14937.0,14937.00
9,client1,20251013,1,1,1,1.00,1.0,16431.0,16431.00,16431.0,16431.00


In [9]:
# View summary statistics
print("Daily Spending Summary:")
attrs[['price_total_sum', 'price_total_mean', 'transaction_mean']].describe()

Daily Spending Summary:


,price_total_sum,price_total_mean,transaction_mean
count,327.000000,327.000000,327.000000
mean,163595.779817,85628.660336,85628.660336
std,159127.050952,72285.124658,72285.124658
min,1418.000000,1418.000000,1418.000000
25%,53027.500000,36752.000000,36752.000000
50%,115789.000000,68332.000000,68332.000000
75%,215852.000000,110522.800000,110522.800000
max,810744.000000,442376.000000,442376.000000


In [10]:
# View activity patterns
print("Daily Activity Patterns:")
print(f"\nAverage products per customer-day: {attrs['product_id_count'].mean():.2f}")
print(f"Average transactions per customer-day: {attrs['trans_id_count'].mean():.2f}")
print(f"Average items per customer-day: {attrs['quantity_sum'].mean():.2f}")
print(f"\nTop 10 highest spending customer-days:")
attrs.nlargest(10, 'price_total_sum')[['in_customer_id', 'dt_date', 'price_total_sum', 'trans_id_count']]

Daily Activity Patterns:

Average products per customer-day: 1.74
Average transactions per customer-day: 1.86
Average items per customer-day: 7.18

Top 10 highest spending customer-days:


,in_customer_id,dt_date,price_total_sum,trans_id_count
302,client8,20251030,810744.0,3
196,client3,20251027,761170.0,3
57,client11,20251022,723582.0,2
277,client7,20251028,699934.0,2
183,client3,20251002,693281.0,4
101,client13,20251017,683048.0,2
184,client3,20251003,680414.0,3
90,client12,20251030,657111.0,6
136,client15,20251003,649825.0,3
97,client13,20251012,646413.0,2


## 8. Export to Excel

In [11]:
# Export model results to Excel
exporter = ExcelExporter(ctx)
output_file = f'outputs/{cfg_model["model_name"]}_export.xlsx'
exporter.export_model(cfg_model['model_name'], output_file, include_input=True)

print(f"✓ Export complete: {output_file}")
print("\nExcel tabs:")
print(f"  1. {cfg_model['input_dataset_name']} (input)")
print(f"  2. {cfg_model['model_name']}_filters")
print(f"  3. {cfg_model['model_name']}_attrs")

✓ Export complete: outputs/customer_daily_export.xlsx

Excel tabs:
  1. transactions_filters (input)
  2. customer_daily_filters
  3. customer_daily_attrs


## 9. Save Context State

Save the complete context state for use in downstream notebooks:

In [12]:
# Save context state (datasets, config, metadata)
state_dir = save_context_state(ctx=ctx, base_cfg=base_cfg)

print(f"✓ Context state saved: {state_dir}")
print(f"  - Total datasets: {len(ctx.datasets)}")
print(f"\nTo load this state in another notebook:")
print(f"  from src.core.persistence import load_context_state")
print(f"  ctx, base_cfg = load_context_state('{state_dir}')")

✓ Context state saved: data\context_states\test_client_20251022_150907
  - Total datasets: 11

To load this state in another notebook:
  from src.core.persistence import load_context_state
  ctx, base_cfg = load_context_state('data\context_states\test_client_20251022_150907')
